In [1]:
import pyodbc
import pandas as pd
import plotly.express as plot

In [2]:

# Connect to the SQL Server database

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};Server=localhost;Database=Instruments;Trusted_Connection=yes"
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [18]:

#Load watchlist prices
df = pd.read_sql("EXECUTE [dbo].[uspWatchlistPrices]", conn).fillna(0)

df = df.drop(df.columns[5:], axis=1)

print(df)

            Date  Ashoka India Equity Investment Ord  \
0     2020-07-21                               105.0   
1     2020-07-22                               105.0   
2     2020-07-23                               105.0   
3     2020-07-24                               104.5   
4     2020-07-27                               104.5   
...          ...                                 ...   
1291  2025-07-07                               281.0   
1292  2025-07-08                               280.0   
1293  2025-07-09                               283.5   
1294  2025-07-10                               284.0   
1295  2025-07-11                               284.0   

      FAST - Global Fund Y-PF-ACC-GBP  Invesco S&P 500 QVM ETF GBP  \
0                          172.520004                    30.989935   
1                          171.479996                    30.896221   
2                          173.300003                    31.139845   
3                          170.179993          

C:\Users\Pandelus\AppData\Local\Temp\ipykernel_71024\1610648237.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\Pandelus\AppData\Local\Temp\ipykernel_71024\1610648237.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [4]:
# Normalise the data
df.set_index('Date', inplace=True)

# Normalize numeric columns to start at 100
df_normalized = df / df.iloc[0] * 100

# Optional: Reset index if you want 'Date' back as a column
df_normalized = df_normalized.reset_index()

print(df_normalized)


            Date  Ashoka India Equity Investment Ord  \
0     2020-07-21                          100.000000   
1     2020-07-22                          100.000000   
2     2020-07-23                          100.000000   
3     2020-07-24                           99.523810   
4     2020-07-27                           99.523810   
...          ...                                 ...   
1291  2025-07-07                          267.619048   
1292  2025-07-08                          266.666667   
1293  2025-07-09                          270.000000   
1294  2025-07-10                          270.476190   
1295  2025-07-11                          270.476190   

      FAST - Global Fund Y-PF-ACC-GBP  Invesco S&P 500 QVM ETF GBP  \
0                          100.000000                   100.000000   
1                           99.397166                    99.697599   
2                          100.452121                   100.483738   
3                           98.643629          

In [19]:
# Add total column

total_col = df_normalized[df_normalized.columns[1:]].sum(axis=1)

df_normalized['Total'] = total_col
print(df_normalized)


            Date  Ashoka India Equity Investment Ord  \
0     2020-07-21                          100.000000   
1     2020-07-22                          100.000000   
2     2020-07-23                          100.000000   
3     2020-07-24                           99.523810   
4     2020-07-27                           99.523810   
...          ...                                 ...   
1291  2025-07-07                          267.619048   
1292  2025-07-08                          266.666667   
1293  2025-07-09                          270.000000   
1294  2025-07-10                          270.476190   
1295  2025-07-11                          270.476190   

      FAST - Global Fund Y-PF-ACC-GBP  Invesco S&P 500 QVM ETF GBP  \
0                          100.000000                   100.000000   
1                           99.397166                    99.697599   
2                          100.452121                   100.483738   
3                           98.643629          

#### Graph results

In [20]:
fig = plot.line(df_normalized, x='Date', y=df.columns[1:], 
                title='Normalized Watchlist Prices',
                labels={'value': 'Price', 'variable': 'Instrument'},
                template='plotly_dark'
                )
fig.update_layout(autosize=False, width=1200, height=600 )
fig.show()